In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('./data/politik_merge.csv')
data.head(5)

,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas
1,Amnesty International Beberkan 6 Indikator Kri...,2024-07-18,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International,Amnesty International Indonesia,Kebebasan Berpendapat,Indeks Demokrasi,Revisi UU TNI,tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",kereta cepat whoosh,stasiun halim,long weekend,NaN,NaN,detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU,pemilu 2024,Hasyim Asy'ari,NaN,NaN,kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,2024-04-29,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Senam Haji,Kemenag,Jemaah Haji,Ibadah Haji,Asrama Haji,tempo


In [2]:
# Drop beberapa kolom yang tidak diperlukan
data = data.drop(columns=['Judul', 'Waktu', 'Link', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'source'])
data.head(5)

,Content
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem..."


In [9]:
import pandas as pd
import re
import string
import numpy as np

def redundant_format(text):
    text = re.sub(r'^[A-Z\s,.-]+ - ', '', text)
    text = re.sub(r'Simak(.*?)(di sini|juga Video.*)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[[^\]]+\]', '', text)
    return text

def remove_media_branding(text):
    # Buang domain dan watermark media
    patterns = [
        r'\bkompas\.com\b', r'\btemp[o|o]\.co\.id\b', r'\bcnnindonesia\.com\b', r'\bdetik\.com\b',
        r'\bliputan6\.com\b', r'\btribunnews\.com\b', r'\bcnbcindonesia\.com\b',
        r'\bviva\.co\.id\b', r'\bokezone\.com\b', r'\bindozone\.id\b', r'\bberitasatu\.com\b',
        r'\bkompascom\b', r'\btempocom\b', r'\bcnnindonesiacom\b', r'\bdetikcom\b',
        r'\bliputan6com\b', r'\btribunnewscom\b', r'\bcnbcindonesiacom\b',
        r'\bviva\b', r'\bokezone\b', r'\bindozone\b', r'\bberitasatu\b'
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

def normalize_punctuation(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip().lower()

def filter_non_narrative(text):
    blacklist = ['tv one', 'antv', 'net tv', 'andromeda mercury', 'dwi anggia']
    if any(word in text.lower() for word in blacklist):
        return np.nan
    return text

def clean_pipeline(text):
    if pd.isna(text): return np.nan
    text = redundant_format(text)
    text = remove_media_branding(text)  # Baru ditambahkan di sini
    text = normalize_punctuation(text)
    text = filter_non_narrative(text)
    return text

In [10]:
# Apply pipeline
data['Cleaning'] = data['Content'].apply(clean_pipeline)

# Cek hasil
data.head(5)

,Content,Cleaning
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",jakarta presiden joko widodo memakai baju ada...
1,"TEMPO.CO, Jakarta - Amnesty International Indo...",tempoco jakarta amnesty international indones...
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",stasiun kereta cepat whoosh di halim jakarta t...
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",jakarta komisi pemilihan umum kpu ri memastik...
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",tempoco jakarta kementerian agama atau kemena...


In [11]:
# Print satu baris untuk memeriksa hasil
print(data['Cleaning'].iloc[0])

jakarta  presiden joko widodo memakai baju adat betawi saat hadir di sidang tahunan mpr ri dan sidang bersama dpr ri dan dpd ri di kompleks perlemen senayan jakarta pada jumat 1682024 baju yang dikenakan presiden yakni pakaian adat bangsawan ujung serong asal betawi pakaian tersebut berwana hitam dan dilengkapi dengan peci hitam menurut deputi ii kepala staf kepresidenan ksp abetnego tarigan pakaian adat betawi yang dikenakan presiden kali ini punya makna terima kasih untuk kota jakarta “presiden memaknai pakaian adat betawi sebagai simbol terima kasih untuk kota jakarta yang sudah resmi menjadi ibu kota sejak tahun 1966” kata abetnego dalam keterangan resmi ksp jumat adapun ibu kota indonesia akan segera berpindah ke ibu kota nusantara ikn kalimantan timur yang dibangun pada era jokowi abetnego mengungkapkan pakaian adat kali ini dipilih sendiri presiden pakaian itu mencerminkan nilainilai kesopanan ketaatan terhadap agama kekuatan dan kebijaksanaan yang disimbolkan dari warna dan mot